# Cartographie des abattoirs de volaille en Pologne

Cette cartographie des abattoirs de volaille en Pologne est basée sur la liste des établissements agréés par l'Inspection vétérinaire centrale (GIS) à la date du 6 juin 2020 et selon les dispositions contenues dans le [règlement européen 853/2004](https://eur-lex.europa.eu/legal-content/FR/TXT/?qid=1503411625502&uri=CELEX:02004R0853-20190726) du 29 avril 2004 fixant des règles spécifiques d'hygiène applicables aux denrées alimentaires d'origine animale.

Au sein de la [section II](https://zywnosc.wetgiw.gov.pl/spi/zatw/index.php?sekcja=3&lng=0) correspondant aux viandes de volaille et de lagomorphes (lapins, lièvres et rongeurs), on ne retiendra que les abattoirs de volaille en raison du très faible nombre d'abattoirs de lagomorphes.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('section2.csv', sep=';')

In [3]:
display(df.head(10))
print('Dimensions du dataframe :', df.shape)

,WNI,województwo,powiat,gmina,nazwa,adres,Kat.,A,L,działalnosc powiązana,uwagi,dec. ke 2014/709
0,02013901,dolnośląskie,bolesławiecki,warta bolesławiecka,Ubojnia Brojlex Spółka Jawna w Iwinach Francis...,"59-720 Raciborowice Górne, Iwiny 36B",SH,D,NaN,NaN,NaN,N
1,NaN,NaN,NaN,NaN,NaN,NaN,CP,D,NaN,NaN,NaN,NaN
2,02020302,dolnośląskie,dzierżoniowski,bielawa,Zakład Przetwórstwa Mięsnego Jerzy Gawrycki,58-260 Bielawa ul. Wincentego Witosa 3,CP,D,NaN,CP(I)MM(V)MP(V)PP(VI),mp,N
3,02053902,dolnośląskie,jaworski,mściwojów,Ubojnia Drobiu FRANEK Spółka z ograniczoną odp...,"59-407 Mściwojów, Targoszyn 79 H",SH,DZ,NaN,NaN,NaN,N
4,NaN,NaN,NaN,NaN,NaN,NaN,CP,DZ,NaN,NaN,NaN,NaN
5,02060202,dolnośląskie,jeleniogórski,karpacz,"PEKROL K.Zemankiewicz, M. Cichocki Sp.J.","58-508 Jelenia Góra, Siedlęcin ul. 58-508 Jel...",CP,D,NaN,CP(I)MM(V)SH(I),mp,N
6,02093902,dolnośląskie,legnicki,prochowice,Ubojnia Drobiu Michał Chomiak,"59-230 Prochowice, Lisowice 75",SH,D,NaN,NaN,NaN,N
7,NaN,NaN,NaN,NaN,NaN,NaN,CP,D,NaN,NaN,NaN,NaN
8,02093903,dolnośląskie,legnicki,prochowice,Ubojnia Drobiu GALLUS Sumicki Spółka Jawna,"59-230 Prochowice, Kawice 85",SH,D,NaN,NaN,NaN,N
9,NaN,NaN,NaN,NaN,NaN,NaN,CP,D,NaN,NaN,NaN,NaN


Dimensions du dataframe : (758, 12)


Champs :
- WNI - estampille
- województwo/powiat/gmina - région/district/commune
- nazwa - nom
- adres - adresse
- Kat. - catégorie SH (abattoir) ou CP (atelier de découpe)
- A - volaille
- L - lagomorphes
- D - en activité
- DZ - activité suspendue
- ZW - agrément conditionnel
- działalnosc powiązana - autres activités (abattage, découpe, hachage, transformation...)
- uwagi - remarques (viande fraîche et/ou produits à base de viande)
- dec. ke 2014/709 - concerné par les mesures de lutte contre la peste porcine africaine

## Nettoyage de données

Pour les besoins de notre cartographie, nous ne garderons que les colonnes WNI, województwo, powiat, nazwa, adres et Kat.. Nous ne conserverons également que les établissements en activité ou bien bénéficiant d'un agrément conditionnel.

In [4]:
# Suppression des colonnes inutiles
df.drop(columns=['gmina', 'L', 'działalnosc powiązana', 'uwagi', 'dec. ke 2014/709'], inplace=True)

In [5]:
# Les établissements qui ont à la fois des activités d'abattage et de découpe apparaissent deux fois
# Identification des doublons
df[df['WNI'].isna()]

,WNI,województwo,powiat,nazwa,adres,Kat.,A
1,NaN,NaN,NaN,NaN,NaN,CP,D
4,NaN,NaN,NaN,NaN,NaN,CP,DZ
7,NaN,NaN,NaN,NaN,NaN,CP,D
9,NaN,NaN,NaN,NaN,NaN,CP,D
11,NaN,NaN,NaN,NaN,NaN,CP,D
...,...,...,...,...,...,...,...
734,NaN,NaN,NaN,NaN,NaN,CP,D
745,NaN,NaN,NaN,NaN,NaN,CP,D
750,NaN,NaN,NaN,NaN,NaN,CP,D
754,NaN,NaN,NaN,NaN,NaN,CP,D


In [6]:
# Suppression des doublons dont l'activité est suspendue
df.drop(index=df[(df['WNI'].isna()) & (df['A'] == 'DZ')].index, inplace=True)

In [7]:
# Création de deux colonnes 'abattage' et 'decoupe' aux valeurs booléennes (True or False)
# à la place de la colonne 'Kat.'
df['abattage'] = False
df['decoupe'] = False
df.loc[df['Kat.'] == 'SH', 'abattage'] = True
df.loc[df['Kat.'] == 'CP', 'decoupe'] = True
df.drop(columns=['Kat.'], inplace=True)

In [8]:
# Remplissage des lignes doublons à partir de la ligne précédente
df.loc[((df['WNI'].isna()) & (df['A'] == 'D') & (df['A'].shift(1) == 'DZ')), ['WNI', 'województwo', 'powiat', 'nazwa', 'adres', 'abattage']] = df[['WNI', 'województwo', 'powiat', 'nazwa', 'adres', 'abattage']].shift(1)
# Suppression des établissements dont l'activité est suspendue
df.drop(index=df[df['A'] == 'DZ'].index, inplace=True)
# Suppression de la colonne indiquant l'état de l'activité
df.drop(columns=['A'], inplace=True)
# Remplissage des lignes doublons restantes à partir de la ligne précédente
df.loc[df['WNI'].isna(), ['WNI', 'województwo', 'powiat', 'nazwa', 'adres', 'abattage']] = df[['WNI', 'województwo', 'powiat', 'nazwa', 'adres', 'abattage']].shift(1)
# Suppression des doublons
df.drop_duplicates(subset=['WNI', 'województwo', 'powiat', 'nazwa', 'adres'], keep='last', ignore_index=True, inplace=True)
# Recherche des éventuelles valeurs manquantes
df[df.isnull().any(axis=1)]

,WNI,województwo,powiat,nazwa,adres,abattage,decoupe


Les données sont désormais propres, nous allons encore renommer les colonnes avant de passer à l'étape de conversion des adresses en coordonnées géographiques.

In [9]:
noms_colonnes = ['estampille', 'region', 'district', 'nom', 'adresse', 'abattage', 'decoupe']
df.columns = noms_colonnes
df.head(10)
print('Dimensions du dataframe nettoyé :', df.shape)

Dimensions du dataframe nettoyé : (534, 7)


## Conversion des adresses en coordonnées géographiques

In [10]:
from geopy.geocoders import GoogleV3
# Ne pas oublier d'insérer la clé API
key = 'secret'
geocoder = GoogleV3(api_key=key, domain='maps.googleapis.com')

In [13]:
from functools import partial
# Géocodage à l'aide de l'API Google
df['point'] = df['adresse'].apply(partial(geocoder.geocode, components={"country": "PL"}))

In [14]:
# Extraction des coordonnées
df['coord'] = df['point'].str[1]

## Création de la carte

In [15]:
import folium
from folium.plugins import MarkerCluster

In [21]:
coord_pologne = (51.9189, 19.1344)
m = folium.Map(location=coord_pologne, zoom_start=6)
mc = MarkerCluster().add_to(m)
df.apply(lambda row:folium.Marker(location=row['coord'], popup=row['nom'], icon=folium.Icon(color='orange')).add_to(mc), axis=1)
m